# Project2_1

Notebook to estimate the robot's movements using the encoders.

In [ ]:
# Insérez ici le numéro de votre équipe
# (en) Insert here your team number
VOTRE_NUMERO_EQUIPE = 0

In [ ]:
import sys


sys.path.append("/opt/ros/noetic/lib/python3/dist-packages/")

In [ ]:
# Import des paquets requis et configuration du ROS Master
# (en) Import of required packages and configuration of the ROS Master
from os import environ
from filterpy.kalman import KalmanFilter
from geometry_msgs.msg import Point, Pose, Quaternion, Twist, Vector3
from nav_msgs.msg import Odometry
from sensor_msgs.msg import Imu
from jackal_msgs.msg import Drive
from jackal_msgs.msg import Feedback
from math import sin, cos
from scipy.spatial.transform import Rotation as R
import rospy
import numpy as np

# from lab_utils.project2 import publish_odom_tf

In [ ]:
environ["ROS_MASTER_URI"] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node("dingo_state_estimation", anonymous=True)

In [ ]:
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# Entrée : Quaternion [x, y ,z ,w]
# Sortie : Angle de lacet (yaw) en radians
# (en) Function for calculating the orientation from a quaternion
# Input: Quaternion [x, y ,z ,w]
# Output: Yaw angle in radians
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler("xyz", degrees=False)
    return angles[2]

In [ ]:
# Ground truth subscriber callback
# (en) Ground truth subscriber callback
ground_truth_msg = Odometry()


def ground_truth_callback(msg):
    global ground_truth_msg
    ground_truth_msg = msg

In [ ]:
# UMI subscriber callback
# Dans cette fonction la variable msg contient les mesures de l'UMI
# (en) UMI subscriber callback
# In this function the variable msg contains the UMI measurements
def imu_callback(msg):
    pass
    # Utiliser les valeurs du IMU dans cette fonction
    # (en) Use the IMU values in this function

### Configuration du script (Script configuration)

Ajoutez ici les données géométriques du robot.

(en) Add here the geometric data of the robot.

In [ ]:
# À compléter selon les tâches précédentes
# (en) To be completed according to the previous tasks
interwheel_distance = 0.0
left_wheel_radius = 0.0
right_wheel_radius = 0.0

### Estimer les déplacements du robot (Estimate the robot's movements)

Dans la prochaine cellule ajoutez votre code pour estimer les variables x, y et theta du robot (et leurs vitesses).

(en) In the next cell add your code to estimate the variables x, y and theta of the robot (and their speeds).

In [ ]:
# Encodeurs subscriber callback
# Estimation de la position du robot selon les mesures des encodeurs
# (en) Encoder subscriber callback
# Estimation of the robot position according to the encoder measurements
prev_left_pos = 0.0
prev_right_pos = 0.0
last_time_ = rospy.Time.now()


def encoders_callback(msg):
    global prev_left_pos, prev_right_pos, last_time_

    # Obtenir dt
    # (en) Get dt
    time = rospy.Time.now()
    dt = (time - last_time_).to_sec()
    if dt < 0.0001:
        return

    # Obtenir les mesures de chaque encodeur
    # (en) Get the measurements of each encoder
    left_pos = msg.drivers[0].measured_travel
    right_pos = msg.drivers[1].measured_travel

    # Ajoutez ici votre code pour estimer les déplacements du robot:
    # (en) Add your code here to estimate the robot displacements:

    # Enregistrez les valeurs pour la prochaine fois
    # (en) Save the values for next time
    prev_left_pos = left_wheel_cur_pos
    prev_right_pos = right_wheel_cur_pos

### Configuration du script

Ajoutez ici les noms des messages ROS

In [ ]:
# ROS subscribers et publishers
# (en) ROS subscribers and publishers
feedback_sub = rospy.Subscriber("XXX", Feedback, encoders_callback)
imu_sub = rospy.Subscriber("XXX", Imu, imu_callback)
ground_truth_sub = rospy.Subscriber("XXX", Odometry, ground_truth_callback)
cmd_drive_pub = rospy.Publisher("XXX", Drive, queue_size=1)

### Tester l'estimation

La prochaine cellule déplace le robot dans une courbe pour 20 secondes et enregistre sa pose réelle la pose estimée dans un rosbag.

Changez les 0 pour les valeurs correspondantes de votre filtre Kalman.

In [ ]:
# Création d'un nouveau contenant ROSBAG (en écriture)
# (en) Creation of a new ROSBAG container (in writing)
import rosbag

test_bag = rosbag.Bag("project2_1.bag", "w")


# Publie l'odométrie et la TF pendant 20 secondes
# (en) Publish odometry and TF for 20 seconds
start = float(rospy.Time().now().secs)

rate = rospy.Rate(50)  # 50hz

while (float(rospy.Time().now().secs) - start) < 20:
    # ***TÂCHE***
    # Changez les 0 pour les valeurs correspondantes de votre estimateur
    # (en) Change the 0 for the corresponding values of your estimator
    x = 0
    y = 0

    vlin = 0

    vang = 0
    theta = 0
    # ***TÂCHE***

    # Déplacer le robot
    # (en) Move the robot
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = 5.0
    cmd_drive_msg.drivers[1] = 7.0
    cmd_drive_pub.publish(cmd_drive_msg)

    # Enregistrer le déplacement du robot
    # (en) Record the robot displacement
    pose = (x, y, 0)

    r = R.from_euler("xyz", [0, 0, theta], degrees=False)

    orientation = r.as_quat()
    odometry_msg = Odometry()
    odometry_msg.header.frame_id = "odom"
    odometry_msg.header.stamp = rospy.Time.now()
    odometry_msg.child_frame_id = "base_link"
    odometry_msg.pose.pose = Pose(Point(*pose), Quaternion(*orientation))
    odometry_msg.twist.twist = Twist(Vector3(vlin, 0, 0), Vector3(0, 0, vang))

    test_bag.write("/filter", odometry_msg, rospy.Time().now())
    test_bag.write("/ground_truth", ground_truth_msg, rospy.Time().now())
    rate.sleep()


# fermer proprement le ROSBAG
# (en) close the ROSBAG properly
test_bag.close()

In [ ]:
# Arreter le dingo
# (en) Stop the dingo
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0
cmd_drive_pub.publish(cmd_drive_msg)